# GridSearch & Pipelines
GridSearch es una herramienta de optimización que usamos cuando ajustamos hiperparámetros. Definimos la cuadrícula(grid) de parámetros que queremos buscar y seleccionamos la mejor combinación de parámetros para nuestros datos.


## Método 1
Itera un único algoritmo sobre un conjunto de hiperparámetros, mediante la validación cruzada, iterando con el dataset dividido en train y val para recoger los errores y evaluar la mejor métrica. 

In [ ]:
# Importamos el módulo warnings para gestionar advertencias
import warnings

# Ignoramos las advertencias de deprecación para mantener limpia la salida
# Esto evita que se muestren mensajes de funciones que están quedando obsoletas
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Calculamos el número total de modelos que GridSearch evaluará
# 4 kernels × 7 valores de C × 7 valores de degree × 2 valores de gamma = 392 combinaciones
print(4*7*7*2, "modelos")

In [ ]:
# Importamos las librerías necesarias para el GridSearch básico
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

# Cargamos el dataset Iris (clasificación de flores con 4 características)
iris = datasets.load_iris()

# Definimos el espacio de búsqueda de hiperparámetros para SVM
parameters = {
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],  # Tipos de kernel a probar
    'C': [0.001, 0.1, 0.5, 1, 5, 10, 100],  # Parámetro de regularización
    'degree': [1,2,3,4,5,6,7],  # Grado del polinomio (solo para kernel 'poly')
    'gamma': ['scale', 'auto']  # Coeficiente del kernel para 'rbf', 'poly' y 'sigmoid'
}

# Creamos una instancia del clasificador SVM sin hiperparámetros específicos
svc = svm.SVC()

# Configuramos GridSearchCV para buscar la mejor combinación de hiperparámetros
clf = GridSearchCV(estimator = svc,  # Modelo a optimizar
                  param_grid = parameters,  # Espacio de búsqueda
                  n_jobs = -1,  # Usar todos los núcleos del procesador
                  cv = 10,  # Validación cruzada con 10 particiones
                  scoring="accuracy")  # Métrica de evaluación

# Entrenamos el modelo probando todas las combinaciones
clf.fit(iris.data, iris.target)

In [ ]:
# Mostramos el mejor estimador (modelo) encontrado por GridSearch
# Este es el modelo con los hiperparámetros óptimos
clf.best_estimator_

In [ ]:
# Mostramos los mejores hiperparámetros encontrados
print(clf.best_params_)

# Mostramos el mejor score (precisión) obtenido con validación cruzada
# Este es el promedio de accuracy en los 10 folds
print(clf.best_score_)

In [ ]:
# Importamos la función para validación cruzada manual
from sklearn.model_selection import cross_val_score

# Creamos un modelo SVM con los mejores hiperparámetros encontrados
clf = svm.SVC(C=0.1, degree=2, gamma='auto', kernel='poly')

# Realizamos validación cruzada con 10 folds para evaluar el modelo
# Esto nos da un score por cada fold (partición de los datos)
scores = cross_val_score(clf, iris.data, iris.target, cv=10)
scores

In [ ]:
# Importamos numpy para cálculos estadísticos
import numpy as np

# Calculamos la media de los scores de validación cruzada
# Esto nos da una estimación de la precisión esperada del modelo
print(np.mean(scores))

# Calculamos la desviación estándar de los scores
# Un valor bajo indica que el modelo es estable entre diferentes particiones
print(np.std(scores))

## Método 2

Una forma más senior es montar un único gridsearch para iterar con varios modelos con otros hiperparámetros y con la validación cruzada.

In [ ]:
# Importamos pickle para guardar y cargar modelos entrenados
# Pickle nos permite serializar objetos de Python a archivos
import pickle

In [ ]:
# Importamos todas las librerías necesarias para el Método 2
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression  # Modelo de regresión logística
from sklearn.ensemble import RandomForestClassifier  # Modelo de bosque aleatorio
from sklearn.model_selection import GridSearchCV  # Búsqueda de hiperparámetros
from sklearn.pipeline import Pipeline  # Para encadenar transformaciones y modelo
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Escaladores de datos
from sklearn.model_selection import train_test_split  # División de datos
from sklearn import svm  # Support Vector Machines

# Establecemos semilla aleatoria para reproducibilidad de resultados
np.random.seed(0)

In [ ]:
# Cargamos el dataset Iris
iris = datasets.load_iris()

# Separamos las características (X) de las etiquetas (y)
X = iris.data  # Matriz de características (150 muestras × 4 características)
y = iris.target  # Vector de etiquetas (0, 1, o 2 para cada especie)

In [ ]:
# Dividimos los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X,  # Características
                                                    y,  # Etiquetas
                                                    test_size=0.2,  # 20% para test
                                                    random_state=2)  # Semilla para reproducibilidad

In [ ]:
# Creamos un Pipeline que encadena preprocesamiento y modelo
# Un pipeline ejecuta pasos secuencialmente: primero escala, luego clasifica
pipe = Pipeline(steps=[("scaler", StandardScaler()),  # Paso 1: Estandarización
    ('classifier', svm.SVC())  # Paso 2: Clasificador
])

# CONFIGURACIÓN 1: Regresión Logística con diferentes iteraciones y penalizaciones
logistic_params = {
    # Probamos dos configuraciones de regresión logística
    'classifier': [LogisticRegression(max_iter=1000, solver='liblinear'), 
                   LogisticRegression(max_iter=10, solver='liblinear')],
    'classifier__penalty': ['l1', 'l2']  # Penalización L1 (Lasso) o L2 (Ridge)
}

# CONFIGURACIÓN 2: Random Forest con diferentes escaladores y profundidades
random_forest_params = {
    'scaler': [StandardScaler(), MinMaxScaler(), None],  # Diferentes escaladores o ninguno
    'classifier': [RandomForestClassifier()],  # Clasificador de bosque aleatorio
    'classifier__max_depth': [2,3]  # Profundidad máxima de los árboles
}

# CONFIGURACIÓN 3: SVM con diferentes valores de C (regularización)
svm_param = {
    'classifier': [svm.SVC()],  # Clasificador SVM
    'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],  # Parámetro de regularización
}

# Creamos una lista con todos los espacios de búsqueda
# GridSearch probará cada uno de estos espacios independientemente
search_space = [
    logistic_params,
    random_forest_params,
    svm_param
]

# Configuramos GridSearchCV para comparar múltiples algoritmos
clf = GridSearchCV(estimator = pipe,  # Pipeline base
                  param_grid = search_space,  # Espacio de búsqueda multi-modelo
                  cv = 5,  # Validación cruzada con 5 folds
                  verbose=2,  # Muestra información del proceso
                  n_jobs=-1)  # Usa todos los núcleos

# Entrenamos todos los modelos y encontramos el mejor
clf.fit(X_train, y_train)

In [ ]:
# Mostramos el pipeline completo del mejor modelo encontrado
print(clf.best_estimator_)

# Mostramos el mejor score obtenido en validación cruzada
print(clf.best_score_)

# Mostramos los mejores parámetros (incluyendo el modelo y sus hiperparámetros)
print(clf.best_params_)

In [ ]:
# Usamos el mejor modelo encontrado para hacer predicciones en el conjunto de test
# Esto nos muestra las clases predichas para cada muestra del test
clf.best_estimator_.predict(X_test)

In [ ]:
# Evaluamos el mejor modelo en el conjunto de test
# Esto nos da la accuracy (precisión) del modelo en datos no vistos durante el entrenamiento
clf.best_estimator_.score(X_test,y_test)

In [ ]:
# Mostramos nuevamente el mejor estimador
# Es el pipeline completo con scaler y clasificador
clf.best_estimator_

## Método 3

Otro uso puede ser la construcción de pipelines (tuberías) específicos para cada tipo de modelo.

In [ ]:
# Importamos todas las librerías necesarias para el Método 3
from sklearn.preprocessing import StandardScaler  # Estandarización de datos
from sklearn.impute import SimpleImputer  # Imputación de valores faltantes
from sklearn.pipeline import Pipeline  # Para crear pipelines
from sklearn.model_selection import GridSearchCV  # Búsqueda de hiperparámetros
from sklearn.feature_selection import SelectKBest  # Selección de mejores características
from sklearn.metrics import accuracy_score  # Métrica de evaluación

import pandas as pd  # Manipulación de datos tabulares
import numpy as np  # Operaciones numéricas

# Importamos los clasificadores que usaremos
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.linear_model import LogisticRegression  # Regresión Logística
from sklearn.ensemble import RandomForestClassifier  # Random Forest

In [ ]:
# PIPELINE 1: Regresión Logística con preprocesamiento completo
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),  # Paso 1: Imputa valores faltantes
    ("scaler", StandardScaler()),  # Paso 2: Estandariza las características
    ("reglog", LogisticRegression())  # Paso 3: Modelo de regresión logística
])

# Espacio de búsqueda para Regresión Logística
reg_log_param = {
    "imputer__strategy": ['mean', 'median'],  # Estrategia de imputación: media o mediana
    "reglog__penalty": ['l1', 'l2'],  # Tipo de regularización
    "reglog__C": np.logspace(0, 4, 10)  # 10 valores de C entre 10^0 y 10^4 (escala logarítmica)
}

In [ ]:
# PIPELINE 2: Random Forest (sin pipeline, modelo directo)
rand_forest = RandomForestClassifier()

# Espacio de búsqueda para Random Forest
rand_forest_param = {
    "n_estimators": [10, 100, 1000],  # Número de árboles en el bosque
    "max_features": [1,2,3]  # Número máximo de características a considerar por división
}

# PIPELINE 3: SVM con preprocesamiento y selección de características
svm = Pipeline(steps=[
    ("scaler", StandardScaler()),  # Paso 1: Estandarización
    ("selectkbest", SelectKBest()),  # Paso 2: Selección de k mejores características
    ("svm", SVC())  # Paso 3: Clasificador SVM
])

# Espacio de búsqueda para SVM
svm_param = {
    'selectkbest__k': [2, 3, 4],  # Número de características a seleccionar
    'svm__kernel': ['linear', 'rbf', 'sigmoid', 'poly'],  # Tipo de kernel
    'svm__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],  # Parámetro de regularización
    'svm__degree': [1,2,3,4],  # Grado del polinomio (solo para kernel 'poly')
    'svm__gamma': ['scale', 'auto']  # Coeficiente del kernel
}

# Creamos GridSearchCV para cada modelo con su respectivo espacio de búsqueda
gs_reg_log = GridSearchCV(reg_log,  # Pipeline de regresión logística
                         reg_log_param,  # Parámetros a buscar
                         cv = 10,  # 10-fold cross validation
                         scoring = 'accuracy',  # Métrica de evaluación
                         verbose = 1,  # Muestra información del proceso
                         n_jobs = -1)  # Usa todos los núcleos

gs_rand_forest = GridSearchCV(rand_forest,  # Modelo Random Forest
                         rand_forest_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

gs_svm = GridSearchCV(svm,  # Pipeline SVM
                         svm_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

# Creamos un diccionario con todos los grid searches para iterar sobre ellos
grids = {"gs_reg_log": gs_reg_log,
        "gs_rand_forest": gs_rand_forest,
        "gs_svm": gs_svm}

In [ ]:
# Importamos train_test_split para dividir los datos
from sklearn.model_selection import train_test_split 

# Preparamos las características y las etiquetas
X = iris.data  # Características del dataset Iris
y = iris.target  # Etiquetas (especies de flores)

# Dividimos en train (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,  # 20% para test
                                                    random_state=42)  # Semilla para reproducibilidad

In [ ]:
# Iteramos sobre cada grid search y lo entrenamos
# Esto evaluará cada modelo con su respectivo espacio de hiperparámetros
for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)  # Entrenamos cada modelo

In [ ]:
# Mostramos los resultados del mejor modelo de Regresión Logística
print(gs_reg_log.best_score_)  # Mejor accuracy en validación cruzada
print(gs_reg_log.best_params_)  # Mejores hiperparámetros encontrados
print(gs_reg_log.best_estimator_)  # Pipeline completo del mejor modelo
print(gs_reg_log.best_estimator_['reglog'])  # Solo el modelo de regresión logística

In [ ]:
# Mostramos los resultados del mejor modelo de Random Forest
print(gs_rand_forest.best_score_)  # Mejor accuracy en validación cruzada
print(gs_rand_forest.best_params_)  # Mejores hiperparámetros encontrados
print(gs_rand_forest.best_estimator_)  # Modelo completo del mejor Random Forest

In [ ]:
# Mostramos los resultados del mejor modelo SVM
print(gs_svm.best_score_)  # Mejor accuracy en validación cruzada
print(gs_svm.best_params_)  # Mejores hiperparámetros encontrados
print(gs_svm.best_estimator_)  # Pipeline completo del mejor modelo
print(gs_svm.best_estimator_['svm'])  # Solo el clasificador SVM

In [ ]:
# Comparamos los scores de todos los modelos
# Creamos una lista de tuplas con el nombre y el mejor score de cada grid
best_grids = [(i, j.best_score_) for i, j in grids.items()]

# Convertimos a DataFrame y ordenamos por mejor score (descendente)
best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids  # Mostramos la tabla comparativa

In [ ]:
# Mostramos el mejor estimador de SVM (el modelo con mejor score en validación)
gs_svm.best_estimator_

In [ ]:
# Hacemos predicciones con el mejor modelo SVM en el conjunto de test
preds = gs_svm.best_estimator_.predict(X_test)

# Calculamos la accuracy comparando predicciones con valores reales
accuracy_score(y_test, preds)

In [ ]:
# Mostramos el mejor pipeline de Regresión Logística
gs_reg_log.best_estimator_

In [ ]:
# Evaluamos el mejor modelo de Regresión Logística en el conjunto de test
preds = gs_reg_log.best_estimator_.predict(X_test)

# Calculamos la accuracy - ¡Obtenemos 100%! El modelo generaliza perfectamente
accuracy_score(y_test, preds)

In [ ]:
# Evaluamos el mejor modelo de Random Forest en el conjunto de test
preds = gs_rand_forest.best_estimator_.predict(X_test)

# Calculamos la accuracy - ¡También 100%! Generaliza perfectamente
accuracy_score(y_test, preds)

 Tanto la regresión logísitca(pipeline) como el random forest son los modelos que mejor generalizan

In [ ]:
# Mostramos nuevamente el mejor modelo SVM
gs_svm.best_estimator_

In [ ]:
# Accedemos solo al clasificador SVM dentro del pipeline
# Usando notación de diccionario ['svm']
gs_svm.best_estimator_['svm']

In [ ]:
# Evaluamos el SVM en el conjunto de test
preds = gs_svm.best_estimator_.predict(X_test)

# Calculamos la accuracy - 96.67%, ligeramente inferior a los otros dos
accuracy_score(y_test, preds)

In [ ]:
# Accedemos al clasificador SVM del pipeline
gs_svm.best_estimator_['svm']

In [ ]:
# CONCLUSIÓN: El mejor modelo es la Regresión Logística
# Guardamos el mejor modelo para usar en producción
best_model = gs_reg_log.best_estimator_

# Evaluamos el modelo final en el conjunto de test
best_model.score(X_test, y_test)  # Accuracy de 100%

In [ ]:
# Mostramos los mejores parámetros del modelo ganador
gs_reg_log.best_params_

In [ ]:
# Confirmamos que el mejor modelo es Regresión Logística
best_model = gs_reg_log.best_estimator_

# Evaluamos nuevamente para confirmar el resultado
best_model.score(X_test, y_test)

In [ ]:
# Mostramos el pipeline completo del mejor modelo
gs_reg_log.best_estimator_

In [ ]:
# Verificamos una vez más el rendimiento del mejor modelo
best_model = gs_reg_log.best_estimator_
best_model.score(X_test, y_test)

In [ ]:
# Pipeline final del modelo ganador
gs_reg_log.best_estimator_

In [ ]:
# Importamos pickle para serialización
import pickle

# Definimos el nombre del archivo donde guardaremos el modelo
filename = 'finished_model'

# Guardamos el modelo en un archivo binario
# 'wb' = write binary (escritura binaria)
with open(filename, 'wb') as archivo_salida:
    pickle.dump(best_model, archivo_salida)  # Serializamos el modelo

In [ ]:
# Cargamos el modelo guardado desde el archivo
# 'rb' = read binary (lectura binaria)
with open(filename, 'rb') as archivo_entrada:
    modelo_importado = pickle.load(archivo_entrada)  # Deserializamos el modelo

In [ ]:
# Verificamos que el modelo cargado funciona correctamente
# Evaluamos en el conjunto de test y multiplicamos por 100 para ver el porcentaje
modelo_importado.score(X_test, y_test)*100  # 100.0% de accuracy

In [ ]:
# Hacemos predicciones con el modelo cargado
# Esto demuestra que el modelo guardado funciona perfectamente
modelo_importado.predict(X_test)

In [ ]:
# Mostramos la estructura completa del modelo importado
# Verificamos que mantiene todos los pasos del pipeline
modelo_importado

In [ ]:
# Código comentado: Ejemplo de cómo usar el modelo con nuevos datos
# modelo_importado.predict(X_new)

Ya hemos escogido modelo gracias a los datos de validación. Ahora habría que entrenar el modelo con TODOS los datos de train.

## RandomSearch
El problema que tiene el GridSearchCV es que computacionalmente es muy costoso cuando el espacio dimensional de los hiperparámetros es grande.

Mediante el RandomSearch no se prueban todas las combinaciones, sino unas cuantas de manera aleatoria. Funciona bien con datasets con pocas features. Incluso [hay papers](https://www.jmlr.org/papers/v13/bergstra12a.html) que aseguran que es más eficiente RandomSearch frente a GridSearch

![imagen](https://miro.medium.com/proxy/1*ZTlQm_WRcrNqL-nLnx6GJA.png)

In [ ]:
# Generamos 100 valores espaciados logarítmicamente entre 10^-2 y 10^4
# Esto es útil para definir rangos de hiperparámetros en escala logarítmica
# Por ejemplo, para el parámetro C en regresión logística
np.logspace(-2, 4, 100)

In [ ]:
# Importamos RandomizedSearchCV para búsqueda aleatoria de hiperparámetros
from sklearn.model_selection import RandomizedSearchCV

# Creamos el pipeline para Regresión Logística
reg_log = Pipeline(steps=[
                          ("imputer",SimpleImputer()),  # Imputación de valores faltantes
                          ("scaler",StandardScaler()),  # Estandarización
                          ("reglog",LogisticRegression(max_iter=100000))  # Modelo con más iteraciones
                         ])

# Definimos el espacio de búsqueda de hiperparámetros
reg_log_param = {    
                 "imputer__strategy": ['mean', 'median', 'most_frequent'],  # 3 estrategias de imputación
                 "reglog__penalty": ["l1","l2"],  # 2 tipos de penalización
                 "reglog__C": np.logspace(-2, 4, 100)  # 100 valores de C
                }
# En total hay 3×2×100 = 600 combinaciones posibles

# Configuramos RandomizedSearchCV
search = RandomizedSearchCV(reg_log,
                           reg_log_param,
                           n_iter = 50,  # Solo probamos 50 combinaciones aleatorias (en lugar de 600)
                           scoring='accuracy',  # Métrica de evaluación
                           n_jobs=-1,  # Usa todos los núcleos
                           cv=10)  # Validación cruzada con 10 folds

# Ejecutamos la búsqueda aleatoria
result = search.fit(X_train, y_train)

# Mostramos los resultados
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
print('Best Estimator: %s' % result.best_estimator_)

In [ ]:
# Mostramos nuevamente los resultados del RandomizedSearchCV
# Best Score: El mejor accuracy obtenido en validación cruzada
print('Best Score: %s' % result.best_score_)

# Best Hyperparameters: Los mejores hiperparámetros encontrados aleatoriamente
print('Best Hyperparameters: %s' % result.best_params_)

# Best Estimator: El pipeline completo con los mejores hiperparámetros
print('Best Estimator: %s' % result.best_estimator_)

In [ ]:
# Código comentado: Ejemplo de cómo crear un DataFrame con metadatos del modelo
# Esto sería útil para documentar y comparar diferentes modelos en un proyecto

# pd.DataFrame({"modelo":filename_model,
#             "notebook":notebook_name,
#             "accuracy":accuracy})